# Work with TMDB + IMDB dataset 
- [Source](https://www.kaggle.com/datasets/alanvourch/tmdb-movies-daily-updates)

**Metadata**

| **#** | **Column**               | **Description**                                                                                     |
|-------|--------------------------|-----------------------------------------------------------------------------------------------------|
| 1     | **id**                   | Unique identifier for the film in the TMDB.                                                        |
| 2     | **title**                | The official title of the movie.                                                                   |
| 3     | **vote_average**         | Average rating of the movie on a scale from 0 to 10.                                               |
| 4     | **vote_count**           | Number of votes that contributed to the movie's rating.                                            |
| 5     | **status**               | The current release status of the movie (e.g., *Released*, *Post-Production*).                     |
| 6     | **release_date**         | The date when the film was officially released.                                                    |
| 7     | **revenue**              | Box office earnings of the movie.                                                                  |
| 8     | **runtime**              | Duration of the movie in minutes.                                                                  |
| 9     | **budget**               | Financial budget allocated for the movie production.                                               |
| 10    | **imdb_id**              | Identifier for the movie in the Internet Movie Database (IMDB).                                    |
| 11    | **original_language**    | The language in which the movie was originally produced.                                           |
| 12    | **original_title**       | The title of the movie in its original language.                                                   |
| 13    | **overview**             | Brief summary of the movie's plot.                                                                 |
| 14    | **popularity**           | Popularity score of the movie on TMDB.                                                             |
| 15    | **tagline**              | Official tagline of the movie.                                                                     |
| 16    | **genres**               | Categories of genres the movie belongs to.                                                         |
| 17    | **production_companies** | Companies involved in producing the movie.                                                        |
| 18    | **production_countries** | Countries where the movie was produced.                                                            |
| 19    | **spoken_languages**     | Languages spoken in the movie.                                                                     |
| 20    | **cast**                 | All cast.                                                                                          |
| 21    | **director**             | All director(s).                                                                                   |
| 22    | **director_of_photography** | All DOP (cinematographers).                                                                       |
| 23    | **writers**              | All writers.                                                                                       |
| 24    | **producers**            | Producers and executive producers.                                                                 |
| 25    | **music_composer**       | Music by.                                                                                          |
| 26    | **imdb_rating**          | IMDB rating.                                                                                       |
| 27    | **imdb_votes**           | IMDB vote count.                                                                                   |
| 28    | **poster_path**          | Path to the movie's poster image.                                                                  |




In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import re
import matplotlib.pyplot as plt
import seaborn as sns
import langcodes
import requests
from dotenv import load_dotenv
import os
from bs4 import BeautifulSoup

In [2]:
import sys
sys.path.append('../utils')
import functions

In [21]:
tmdb_df = pd.read_csv('../data/local/raw/TMDB_all_movies.csv')

In [22]:
functions.show_basic_info(tmdb_df)


DataFrame Shape: (1028026, 28)
Number of Rows: 1028026
Number of Columns: 28

Data Types of Columns:
id                           int64
title                       object
vote_average               float64
vote_count                 float64
status                      object
release_date                object
revenue                    float64
runtime                    float64
budget                     float64
imdb_id                     object
original_language           object
original_title              object
overview                    object
popularity                 float64
tagline                     object
genres                      object
production_companies        object
production_countries        object
spoken_languages            object
cast                        object
director                    object
director_of_photography     object
writers                     object
producers                   object
music_composer              object
imdb_rating            

Columns to drop:
- cast
- director_of_photography
- music_composer
- poster_path
- writers
- tagline

In [23]:
tmdb_df.drop(columns=['cast', 'director_of_photography', 'music_composer', 'poster_path', 'writers', 'tagline'], inplace=True)
# tmdb_df.head()

In [24]:
functions.check_for_duplicates(tmdb_df)


No duplicate rows found in the DataFrame.


Remove all rows where 'status' is not 'Released'

In [25]:
print(tmdb_df['status'].unique())

['Released' 'Rumored' 'Post Production' 'Canceled' 'Planned'
 'In Production' nan]


In [26]:
initial_rows = len(tmdb_df)
tmdb_df = tmdb_df[tmdb_df['status'] == 'Released'] # keep rows where 'status' is 'Released'
final_rows = len(tmdb_df)
removed_rows = initial_rows - final_rows
print(f'Number of rows removed: {removed_rows}')

Number of rows removed: 16758


#### 'release_date' column
- Convert to datetime
- Extract year only
- Convert year to integer

In [27]:
df = tmdb_df.copy()

In [28]:
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['release_year'] = df['release_date'].dt.year.astype('Int64')
print(df[['release_date', 'release_year']].head())

  release_date  release_year
0   1988-10-21          1988
1   1986-10-17          1986
2   1995-12-09          1995
3   1993-10-15          1993
4   2006-01-01          2006


Get released movie count per year in df

In [29]:
release_year_counts = df['release_year'].value_counts().sort_index()
print(release_year_counts)

release_year
1800        1
1865        1
1874        1
1878       31
1882        1
        ...  
2022    41950
2023    47546
2024    44390
2025      194
2026        1
Name: count, Length: 147, dtype: Int64


Remove all movies before 1906 and after 2019

In [30]:
# Number of rows before filtering
initial_count = len(df)

# Filter out rows to keep only movies made between 1906 and 2018 (inclusive)
df = df[(df['release_year'] >= 1906) & (df['release_year'] <= 2018)]

# Number of rows after filtering
final_count = len(df)

# Calculate and print the number of rows dropped
rows_dropped = initial_count - final_count
print(f"Number of rows dropped: {rows_dropped}")

# Print the filtered DataFrame
print(df)

Number of rows dropped: 351200
              id                             title  vote_average  vote_count  \
0              2                             Ariel           7.1       335.0   
1              3               Shadows in Paradise           7.3       369.0   
2              5                        Four Rooms           5.8      2628.0   
3              6                    Judgment Night           6.5       331.0   
4              8  Life in Loops (A Megacities RMX)           7.5        27.0   
...          ...                               ...           ...         ...   
1027914  1400054         Ola: Health is Everything           0.0         0.0   
1027917  1400057                           Turksib           0.0         0.0   
1027924  1400064                         Lovelance           0.0         0.0   
1027926  1400068                        Passengers           0.0         0.0   
1028009  1400165           Sesame Street at Night?           0.0         0.0   

        

In [31]:
release_year_counts = df['release_year'].value_counts().sort_index()
print(release_year_counts)

release_year
1906      332
1907      326
1908      432
1909      542
1910      818
        ...  
2014    25310
2015    26430
2016    27813
2017    31033
2018    33181
Name: count, Length: 113, dtype: Int64


In [32]:
df.drop(['overview', 'production_companies', 'production_countries', 'producers', 'status', 'spoken_languages'], axis=1, inplace=True)

In [33]:
df.head()

,id,title,vote_average,vote_count,release_date,revenue,runtime,budget,imdb_id,original_language,original_title,popularity,genres,director,imdb_rating,imdb_votes,release_year
0,2,Ariel,7.1,335.0,1988-10-21,0.0,73.0,0.0,tt0094675,fi,Ariel,11.915,"Comedy, Drama, Romance, Crime",Aki Kaurismäki,7.4,8812.0,1988
1,3,Shadows in Paradise,7.3,369.0,1986-10-17,0.0,74.0,0.0,tt0092149,fi,Varjoja paratiisissa,16.287,"Comedy, Drama, Romance",Aki Kaurismäki,7.5,7587.0,1986
2,5,Four Rooms,5.8,2628.0,1995-12-09,4257354.0,98.0,4000000.0,tt0113101,en,Four Rooms,21.312,Comedy,"Quentin Tarantino, Robert Rodriguez, Alexandre...",6.7,112798.0,1995
3,6,Judgment Night,6.5,331.0,1993-10-15,12136938.0,109.0,21000000.0,tt0107286,en,Judgment Night,8.924,"Action, Crime, Thriller",Stephen Hopkins,6.6,19361.0,1993
4,8,Life in Loops (A Megacities RMX),7.5,27.0,2006-01-01,0.0,80.0,42000.0,tt0825671,en,Life in Loops (A Megacities RMX),3.203,Documentary,Timo Novotny,8.2,284.0,2006


In [34]:
functions.show_missing_values(df)


Missing Values in Columns:
id                        0
title                     2
vote_average              0
vote_count                0
release_date              0
revenue                   0
runtime                   0
budget                    0
imdb_id              205477
original_language         0
original_title            2
popularity                0
genres               161392
director              96986
imdb_rating          310687
imdb_votes           310687
release_year              0
dtype: int64


In [35]:
# drop missing titles
df.dropna(subset=['title'], inplace=True)


In [36]:
display(df)

,id,title,vote_average,vote_count,release_date,revenue,runtime,budget,imdb_id,original_language,original_title,popularity,genres,director,imdb_rating,imdb_votes,release_year
0,2,Ariel,7.1,335.0,1988-10-21,0.0,73.0,0.0,tt0094675,fi,Ariel,11.915,"Comedy, Drama, Romance, Crime",Aki Kaurismäki,7.4,8812.0,1988
1,3,Shadows in Paradise,7.3,369.0,1986-10-17,0.0,74.0,0.0,tt0092149,fi,Varjoja paratiisissa,16.287,"Comedy, Drama, Romance",Aki Kaurismäki,7.5,7587.0,1986
2,5,Four Rooms,5.8,2628.0,1995-12-09,4257354.0,98.0,4000000.0,tt0113101,en,Four Rooms,21.312,Comedy,"Quentin Tarantino, Robert Rodriguez, Alexandre...",6.7,112798.0,1995
3,6,Judgment Night,6.5,331.0,1993-10-15,12136938.0,109.0,21000000.0,tt0107286,en,Judgment Night,8.924,"Action, Crime, Thriller",Stephen Hopkins,6.6,19361.0,1993
4,8,Life in Loops (A Megacities RMX),7.5,27.0,2006-01-01,0.0,80.0,42000.0,tt0825671,en,Life in Loops (A Megacities RMX),3.203,Documentary,Timo Novotny,8.2,284.0,2006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027914,1400054,Ola: Health is Everything,0.0,0.0,2013-04-06,0.0,69.0,50000.0,tt2848736,en,Ola: Health is Everything,0.000,Documentary,Matthew Nagato,NaN,NaN,2013
1027917,1400057,Turksib,0.0,0.0,1986-05-12,0.0,73.0,0.0,tt0275745,ru,На перевале,0.000,Drama,"Serik Zharmukhamedov, Kadyr Zhetpisbay",NaN,NaN,1986
1027924,1400064,Lovelance,0.0,0.0,2018-12-07,0.0,92.0,0.0,tt8374200,fa,عشقولانس,0.000,Comedy,Mohsen Mahini,2.4,177.0,2018
1027926,1400068,Passengers,0.0,0.0,2009-12-14,0.0,0.0,0.0,NaN,fa,مسافران,0.000,"Comedy, Science Fiction",NaN,NaN,NaN,2009


In [37]:
# clean genres
df['genres'] = functions.clean_genres(df, 'genres')
df.head()

,id,title,vote_average,vote_count,release_date,revenue,runtime,budget,imdb_id,original_language,original_title,popularity,genres,director,imdb_rating,imdb_votes,release_year
0,2,Ariel,7.1,335.0,1988-10-21,0.0,73.0,0.0,tt0094675,fi,Ariel,11.915,"comedy, drama, romance, crime",Aki Kaurismäki,7.4,8812.0,1988
1,3,Shadows in Paradise,7.3,369.0,1986-10-17,0.0,74.0,0.0,tt0092149,fi,Varjoja paratiisissa,16.287,"comedy, drama, romance",Aki Kaurismäki,7.5,7587.0,1986
2,5,Four Rooms,5.8,2628.0,1995-12-09,4257354.0,98.0,4000000.0,tt0113101,en,Four Rooms,21.312,comedy,"Quentin Tarantino, Robert Rodriguez, Alexandre...",6.7,112798.0,1995
3,6,Judgment Night,6.5,331.0,1993-10-15,12136938.0,109.0,21000000.0,tt0107286,en,Judgment Night,8.924,"action, crime, thriller",Stephen Hopkins,6.6,19361.0,1993
4,8,Life in Loops (A Megacities RMX),7.5,27.0,2006-01-01,0.0,80.0,42000.0,tt0825671,en,Life in Loops (A Megacities RMX),3.203,documentary,Timo Novotny,8.2,284.0,2006


In [38]:
columns_to_convert = ['imdb_votes', 'revenue', 'budget', 'runtime', 'vote_count']

# Check if the columns exist in the DataFrame
columns_to_convert = [col for col in columns_to_convert if col in df.columns]

# Convert the specified columns to 'Int64' type, handling errors gracefully
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce').astype('Int64')

# Now the DataFrame should have the correct data types
print(df.head())

   id                             title  vote_average  vote_count  \
0   2                             Ariel           7.1         335   
1   3               Shadows in Paradise           7.3         369   
2   5                        Four Rooms           5.8        2628   
3   6                    Judgment Night           6.5         331   
4   8  Life in Loops (A Megacities RMX)           7.5          27   

  release_date   revenue  runtime    budget    imdb_id original_language  \
0   1988-10-21         0       73         0  tt0094675                fi   
1   1986-10-17         0       74         0  tt0092149                fi   
2   1995-12-09   4257354       98   4000000  tt0113101                en   
3   1993-10-15  12136938      109  21000000  tt0107286                en   
4   2006-01-01         0       80     42000  tt0825671                en   

                     original_title  popularity  \
0                             Ariel      11.915   
1              Varjoja par

In [39]:
df.drop(columns='release_date', inplace=True)

In [40]:
df['language'] = df['original_language'].apply(functions.get_language_name)

df.head()

,id,title,vote_average,vote_count,revenue,runtime,budget,imdb_id,original_language,original_title,popularity,genres,director,imdb_rating,imdb_votes,release_year,language
0,2,Ariel,7.1,335,0,73,0,tt0094675,fi,Ariel,11.915,"comedy, drama, romance, crime",Aki Kaurismäki,7.4,8812,1988,Finnish
1,3,Shadows in Paradise,7.3,369,0,74,0,tt0092149,fi,Varjoja paratiisissa,16.287,"comedy, drama, romance",Aki Kaurismäki,7.5,7587,1986,Finnish
2,5,Four Rooms,5.8,2628,4257354,98,4000000,tt0113101,en,Four Rooms,21.312,comedy,"Quentin Tarantino, Robert Rodriguez, Alexandre...",6.7,112798,1995,English
3,6,Judgment Night,6.5,331,12136938,109,21000000,tt0107286,en,Judgment Night,8.924,"action, crime, thriller",Stephen Hopkins,6.6,19361,1993,English
4,8,Life in Loops (A Megacities RMX),7.5,27,0,80,42000,tt0825671,en,Life in Loops (A Megacities RMX),3.203,documentary,Timo Novotny,8.2,284,2006,English


Filter genres

In [41]:
def get_unique_genres(df, column_name='genres'):
    unique_genres = set()  # To store unique genres

    for genres in df[column_name].dropna():  # Drop NaN values
        if isinstance(genres, list):  # If the value is a list
            unique_genres.update([genre.strip() for genre in genres])
        elif isinstance(genres, str):  # If the value is a comma-separated string
            unique_genres.update([genre.strip() for genre in genres.split(',')])

    return unique_genres

# Get unique genres
unique_genres = get_unique_genres(df, column_name='genres')

# Print the unique genres
print("Unique genres:")
print(sorted(unique_genres))  # Sort the genres alphabetically for easier readability


Unique genres:
['action', 'adventure', 'animation', 'comedy', 'crime', 'documentary', 'drama', 'family', 'fantasy', 'history', 'horror', 'music', 'mystery', 'romance', 'science fiction', 'thriller', 'tv movie', 'war', 'western']


In [42]:
def drop_rows_with_specific_genres(df, column_name='genres', genres_to_exclude=None):

    if genres_to_exclude is None:
        genres_to_exclude = {"documentary", "music"}

    # Function to check if a genre exists in the cell
    def contains_excluded_genre(genres):
        if isinstance(genres, list):  # If genres is a list
            return any(genre.strip().lower() in genres_to_exclude for genre in genres)
        elif isinstance(genres, str):  # If genres is a comma-separated string
            return any(genre.strip().lower() in genres_to_exclude for genre in genres.split(','))
        return False  # For NaN or other invalid cases

    # Filter the DataFrame to exclude rows with specific genres
    rows_before = len(df)  # Number of rows before filtering
    filtered_df = df[~df[column_name].apply(contains_excluded_genre)]
    rows_after = len(filtered_df)  # Number of rows after filtering

    # Print how many rows have been dropped
    rows_dropped = rows_before - rows_after
    print(f"Number of rows dropped: {rows_dropped}")

    return filtered_df

# Apply the function
df = drop_rows_with_specific_genres(df, column_name='genres')

# Print the result
print("Filtered DataFrame:")
print(df)


Number of rows dropped: 144208
Filtered DataFrame:
              id                    title  vote_average  vote_count   revenue  \
0              2                    Ariel         7.100         335         0   
1              3      Shadows in Paradise         7.300         369         0   
2              5               Four Rooms         5.800        2628   4257354   
3              6           Judgment Night         6.500         331  12136938   
5              9         Sunday in August         7.135          26         0   
...          ...                      ...           ...         ...       ...   
1027911  1400051              Chilly Days         0.000           0         0   
1027917  1400057                  Turksib         0.000           0         0   
1027924  1400064                Lovelance         0.000           0         0   
1027926  1400068               Passengers         0.000           0         0   
1028009  1400165  Sesame Street at Night?         0.000   

In [43]:
def drop_rows_by_runtime(df, column_name='runtime', min_runtime=40):
    """
    Drops rows where the specified column contains a runtime less than the specified minimum runtime
    and prints how many rows were dropped.

    Parameters:
    -----------
    df : pandas.DataFrame
        The DataFrame to process.
    column_name : str
        The name of the column containing runtime values.
    min_runtime : int
        The minimum runtime value. Rows with runtime less than this value will be dropped.

    Returns:
    --------
    pandas.DataFrame
        A new DataFrame with rows having runtime less than the specified value removed.
    """
    # Ensure that 'runtime' column is treated as integers (convert non-numeric to NaN)
    df[column_name] = pd.to_numeric(df[column_name], errors='coerce')

    # Filter the DataFrame to exclude rows with runtime less than min_runtime
    rows_before = len(df)  # Number of rows before filtering
    filtered_df = df[df[column_name] >= min_runtime]  # Keep rows with runtime >= min_runtime
    rows_after = len(filtered_df)  # Number of rows after filtering

    # Print how many rows have been dropped
    rows_dropped = rows_before - rows_after
    print(f"Number of rows dropped (runtime < {min_runtime}): {rows_dropped}")

    return filtered_df

# Apply the function to drop rows with runtime less than 40
df = drop_rows_by_runtime(df, column_name='runtime', min_runtime=40)

# Print the result
print("Filtered DataFrame:")
print(df)


Number of rows dropped (runtime < 40): 245320
Filtered DataFrame:
              id                               title  vote_average  \
0              2                               Ariel         7.100   
1              3                 Shadows in Paradise         7.300   
2              5                          Four Rooms         5.800   
3              6                      Judgment Night         6.500   
6             11                           Star Wars         8.203   
...          ...                                 ...           ...   
1027797  1399927  The Honeymooners Christmas Special         0.000   
1027869  1400003                            Fly High         0.000   
1027897  1400034                  The Peony Pavilion         0.000   
1027917  1400057                             Turksib         0.000   
1027924  1400064                           Lovelance         0.000   

         vote_count    revenue  runtime    budget    imdb_id  \
0               335          

In [44]:
# def drop_empty_vote_columns_and_count_dropped(df):
#     """
#     Drop rows where 'vote_count' or 'imdb_votes' is empty (NaN or empty string) and return the count of dropped rows.
#     """
#     # Count the rows where 'vote_count' or 'imdb_votes' is NaN or empty
#     rows_before = df.shape[0]
#     rows_to_drop = df[df['vote_count'].isna() | (df['vote_count'] == '') | 
#                       df['imdb_votes'].isna() | (df['imdb_votes'] == '')]
#     rows_to_drop_count = rows_to_drop.shape[0]

#     # Drop the rows with empty 'vote_count' or 'imdb_votes'
#     df_cleaned = df.dropna(subset=['vote_count', 'imdb_votes'])
#     df_cleaned = df_cleaned[df_cleaned['vote_count'] != '']
#     df_cleaned = df_cleaned[df_cleaned['imdb_votes'] != '']

#     # Return the number of dropped rows
#     print(f"Number of rows dropped: {rows_to_drop_count}")
#     return df_cleaned, rows_to_drop_count

# # Example usage:
# # Assuming 'df' is your initial DataFrame with movie data
# df, dropped_rows = drop_empty_vote_columns_and_count_dropped(df)

# # Print the result
# # print(f"Updated DataFrame shape: {df.shape}")
# print(f"Rows dropped: {dropped_rows}")


In [45]:
# Filter rows where the title contains 'vixen' (case-insensitive)
filtered_rows = df[df['title'].str.contains('vixen', case=False, na=False)]

print(filtered_rows)

             id                                              title  \
2952       5157             Beneath the Valley of the Ultra-Vixens   
3162       5706                      Playboy: Voluptuous Vixens II   
3174       5721                                             Vixen!   
3178       5725                                        Supervixens   
18334     29669                                      Video Vixens!   
24070     38211                                       Cycle Vixens   
40618     58951                          Vampire Vixens from Venus   
40619     58952                                     Vampire Vixens   
45228     65181                       Ninja Vixens: Crimson Blades   
45229     65182                   Ninja Vixens: Demonic Sacrifices   
45230     65183                      Ninja Vixens: Devilish Angels   
45231     65184                   Ninja Vixens: Forbidden Paradise   
45853     66036                    Tokyo Train Girls 2: Supervixen   
55877     79054  The

In [46]:
# Count initial rows
initial_count = len(df)

# Remove rows where the title contains 'vixen' (case-insensitive)
df = df[~df['title'].str.contains('vixen', case=False, na=False)]

# Count remaining rows
filtered_count = len(df)

# Calculate dropped rows count
dropped_rows_count = initial_count - filtered_count

print(f"Dropped rows count: {dropped_rows_count}")

Dropped rows count: 47


In [47]:
# Count initial rows
initial_count = len(df)

# Remove rows where the title contains 'vixen' (case-insensitive)
df = df[~df['title'].str.contains('playboy', case=False, na=False)]

# Count remaining rows
filtered_count = len(df)

# Calculate dropped rows count
dropped_rows_count = initial_count - filtered_count

print(f"Dropped rows count: {dropped_rows_count}")

Dropped rows count: 88


In [48]:
import re

def dash_title(title):
    """
    Cleans a single title by removing leading/trailing spaces, normalizing spaces,
    removing special characters, and converting the text to lowercase. Spaces are replaced
    with hyphens.

    Parameters:
    -----------
    title : str
        The title string to be cleaned.

    Returns:
    --------
    str or None
        The cleaned title with hyphens instead of spaces, or None if the input is not a string.
    """
    if isinstance(title, str):  # Check if the title is a string
        title = title.strip()  # Remove leading/trailing spaces
        title = re.sub(r'\s+', ' ', title)  # Replace multiple spaces with a single space
        # Preserve Latin characters and spaces while removing other special characters
        title = re.sub(
            r'[^\w\sàáâäãåçèéêëìíîïñòóôöõùúûüýÿÀÁÂÄÃÅÇÈÉÊËÌÍÎÏÑÒÓÔÖÕÙÚÛÜÝ]', '', title
        )
        title = title.lower()  # Convert the title to lowercase
        title = title.replace(' ', '-')  # Replace spaces with hyphens
        return title
    return None  # Return None for non-string values

# Apply the function to the 'title' column
df['dash_title'] = df['title'].apply(dash_title)

In [49]:
def drop_ufc_titles(df, prefix="ufc-"):
    # Filter rows where 'dash_title' starts with the specified prefix
    rows_before = len(df)  # Number of rows before filtering
    rows_to_drop = df[df['dash_title'].str.startswith(prefix, na=False)]  # Get rows that start with the prefix
    filtered_df = df[~df['dash_title'].str.startswith(prefix, na=False)]  # Keep rows that do not start with the prefix
    rows_after = len(filtered_df)  # Number of rows after filtering

    # Print how many rows have been dropped
    rows_dropped = rows_before - rows_after
    print(f"Number of rows dropped (dash_title starts with '{prefix}'): {rows_dropped}")

    return filtered_df

# Apply the function to drop rows where 'dash_title' starts with 'ufc-'
df = drop_ufc_titles(df, prefix="ufc-")

# Print the result
print("Filtered DataFrame:")
print(df)


Number of rows dropped (dash_title starts with 'ufc-'): 271
Filtered DataFrame:
              id                               title  vote_average  \
0              2                               Ariel         7.100   
1              3                 Shadows in Paradise         7.300   
2              5                          Four Rooms         5.800   
3              6                      Judgment Night         6.500   
6             11                           Star Wars         8.203   
...          ...                                 ...           ...   
1027797  1399927  The Honeymooners Christmas Special         0.000   
1027869  1400003                            Fly High         0.000   
1027897  1400034                  The Peony Pavilion         0.000   
1027917  1400057                             Turksib         0.000   
1027924  1400064                           Lovelance         0.000   

         vote_count    revenue  runtime    budget    imdb_id  \
0              

In [50]:
def drop_baby_einstein_titles(df, prefix="baby-einstein"):
    # Filter rows where 'dash_title' starts with the specified prefix
    rows_before = len(df)  # Number of rows before filtering
    rows_to_drop = df[df['dash_title'].str.startswith(prefix, na=False)]  # Get rows that start with the prefix
    filtered_df = df[~df['dash_title'].str.startswith(prefix, na=False)]  # Keep rows that do not start with the prefix
    rows_after = len(filtered_df)  # Number of rows after filtering

    # Print how many rows have been dropped
    rows_dropped = rows_before - rows_after
    print(f"Number of rows dropped (dash_title starts with '{prefix}'): {rows_dropped}")

    return filtered_df

# Apply the function to drop rows where 'dash_title' starts with 'baby-einstein'
df = drop_baby_einstein_titles(df, prefix="baby-einstein")

# Print the result
print("Filtered DataFrame:")
print(df)


Number of rows dropped (dash_title starts with 'baby-einstein'): 3
Filtered DataFrame:
              id                               title  vote_average  \
0              2                               Ariel         7.100   
1              3                 Shadows in Paradise         7.300   
2              5                          Four Rooms         5.800   
3              6                      Judgment Night         6.500   
6             11                           Star Wars         8.203   
...          ...                                 ...           ...   
1027797  1399927  The Honeymooners Christmas Special         0.000   
1027869  1400003                            Fly High         0.000   
1027897  1400034                  The Peony Pavilion         0.000   
1027917  1400057                             Turksib         0.000   
1027924  1400064                           Lovelance         0.000   

         vote_count    revenue  runtime    budget    imdb_id  \
0       

In [51]:
def drop_wec_titles(df, prefix="wec-"):
    # Filter rows where 'dash_title' starts with the specified prefix
    rows_before = len(df)  # Number of rows before filtering
    rows_to_drop = df[df['dash_title'].str.startswith(prefix, na=False)]  # Get rows that start with the prefix
    filtered_df = df[~df['dash_title'].str.startswith(prefix, na=False)]  # Keep rows that do not start with the prefix
    rows_after = len(filtered_df)  # Number of rows after filtering

    # Print how many rows have been dropped
    rows_dropped = rows_before - rows_after
    print(f"Number of rows dropped (dash_title starts with '{prefix}'): {rows_dropped}")

    return filtered_df

# Apply the function to drop rows where 'dash_title' starts with 'wec-'
df = drop_wec_titles(df, prefix="wec-")

# Print the result
print("Filtered DataFrame:")
print(df)


Number of rows dropped (dash_title starts with 'wec-'): 24
Filtered DataFrame:
              id                               title  vote_average  \
0              2                               Ariel         7.100   
1              3                 Shadows in Paradise         7.300   
2              5                          Four Rooms         5.800   
3              6                      Judgment Night         6.500   
6             11                           Star Wars         8.203   
...          ...                                 ...           ...   
1027797  1399927  The Honeymooners Christmas Special         0.000   
1027869  1400003                            Fly High         0.000   
1027897  1400034                  The Peony Pavilion         0.000   
1027917  1400057                             Turksib         0.000   
1027924  1400064                           Lovelance         0.000   

         vote_count    revenue  runtime    budget    imdb_id  \
0               

In [52]:
# # Count initial rows
# initial_count = len(df)

# # Remove rows where vote_average or vote_count is 0
# df = df[(df['vote_average'] != 0) & (df['vote_count'] != 0)]

# # Count remaining rows
# filtered_count = len(df)

# # Calculate dropped rows count
# dropped_rows_count = initial_count - filtered_count

# print(f"Dropped rows count: {dropped_rows_count}")


In [53]:
def drop_live_at_titles(df):
    # Identify rows where 'dash_title' contains 'live-at-'
    rows_to_drop = df[df['dash_title'].str.contains('live-at-', na=False)]

    # Count the number of dropped rows
    rows_dropped = rows_to_drop.shape[0]
    
    # Drop the identified rows from the DataFrame
    df_cleaned = df[~df['dash_title'].str.contains('live-at-', na=False)]

    # Print the number of rows dropped
    print(f"Dropped {rows_dropped} rows containing 'live-at-' in 'dash_title'")

    return df_cleaned, rows_dropped

# Example usage
df, dropped_count = drop_live_at_titles(df)

# Print the cleaned DataFrame
print(df)


Dropped 439 rows containing 'live-at-' in 'dash_title'
              id                               title  vote_average  \
0              2                               Ariel         7.100   
1              3                 Shadows in Paradise         7.300   
2              5                          Four Rooms         5.800   
3              6                      Judgment Night         6.500   
6             11                           Star Wars         8.203   
...          ...                                 ...           ...   
1027797  1399927  The Honeymooners Christmas Special         0.000   
1027869  1400003                            Fly High         0.000   
1027897  1400034                  The Peony Pavilion         0.000   
1027917  1400057                             Turksib         0.000   
1027924  1400064                           Lovelance         0.000   

         vote_count    revenue  runtime    budget    imdb_id  \
0               335          0       73 

In [54]:
def drop_wwe_titles(df):
    # Identify rows where 'dash_title' contains 'live-at-'
    rows_to_drop = df[df['dash_title'].str.contains('wwe-', na=False)]

    # Count the number of dropped rows
    rows_dropped = rows_to_drop.shape[0]
    
    # Drop the identified rows from the DataFrame
    df_cleaned = df[~df['dash_title'].str.contains('wwe-', na=False)]

    # Print the number of rows dropped
    print(f"Dropped {rows_dropped} rows containing 'wwe-' in 'dash_title'")

    return df_cleaned, rows_dropped

# Example usage
df, dropped_count = drop_wwe_titles(df)

# Print the cleaned DataFrame
print(df)


Dropped 569 rows containing 'wwe-' in 'dash_title'
              id                               title  vote_average  \
0              2                               Ariel         7.100   
1              3                 Shadows in Paradise         7.300   
2              5                          Four Rooms         5.800   
3              6                      Judgment Night         6.500   
6             11                           Star Wars         8.203   
...          ...                                 ...           ...   
1027797  1399927  The Honeymooners Christmas Special         0.000   
1027869  1400003                            Fly High         0.000   
1027897  1400034                  The Peony Pavilion         0.000   
1027917  1400057                             Turksib         0.000   
1027924  1400064                           Lovelance         0.000   

         vote_count    revenue  runtime    budget    imdb_id  \
0               335          0       73     

In [55]:
def drop_championship_titles(df):
    # Identify rows where 'dash_title' contains 'live-at-'
    rows_to_drop = df[df['dash_title'].str.contains('championship', na=False)]

    # Count the number of dropped rows
    rows_dropped = rows_to_drop.shape[0]
    
    # Drop the identified rows from the DataFrame
    df_cleaned = df[~df['dash_title'].str.contains('championship', na=False)]

    # Print the number of rows dropped
    print(f"Dropped {rows_dropped} rows containing 'championship' in 'dash_title'")

    return df_cleaned, rows_dropped

# Example usage
df, dropped_count = drop_championship_titles(df)

# Print the cleaned DataFrame
print(df)


Dropped 89 rows containing 'championship' in 'dash_title'
              id                               title  vote_average  \
0              2                               Ariel         7.100   
1              3                 Shadows in Paradise         7.300   
2              5                          Four Rooms         5.800   
3              6                      Judgment Night         6.500   
6             11                           Star Wars         8.203   
...          ...                                 ...           ...   
1027797  1399927  The Honeymooners Christmas Special         0.000   
1027869  1400003                            Fly High         0.000   
1027897  1400034                  The Peony Pavilion         0.000   
1027917  1400057                             Turksib         0.000   
1027924  1400064                           Lovelance         0.000   

         vote_count    revenue  runtime    budget    imdb_id  \
0               335          0       

In [56]:
def drop_standup_titles(df):
    # Identify rows where 'dash_title' contains 'live-at-'
    rows_to_drop = df[df['dash_title'].str.contains('standup', na=False)]

    # Count the number of dropped rows
    rows_dropped = rows_to_drop.shape[0]
    
    # Drop the identified rows from the DataFrame
    df_cleaned = df[~df['dash_title'].str.contains('standup', na=False)]

    # Print the number of rows dropped
    print(f"Dropped {rows_dropped} rows containing 'standup' in 'dash_title'")

    return df_cleaned, rows_dropped

# Example usage
df, dropped_count = drop_standup_titles(df)

# Print the cleaned DataFrame
print(df)


Dropped 27 rows containing 'standup' in 'dash_title'
              id                               title  vote_average  \
0              2                               Ariel         7.100   
1              3                 Shadows in Paradise         7.300   
2              5                          Four Rooms         5.800   
3              6                      Judgment Night         6.500   
6             11                           Star Wars         8.203   
...          ...                                 ...           ...   
1027797  1399927  The Honeymooners Christmas Special         0.000   
1027869  1400003                            Fly High         0.000   
1027897  1400034                  The Peony Pavilion         0.000   
1027917  1400057                             Turksib         0.000   
1027924  1400064                           Lovelance         0.000   

         vote_count    revenue  runtime    budget    imdb_id  \
0               335          0       73   

In [38]:
# def drop_zero_revenue_budget_and_count_dropped(df):
#     """
#     Drop rows where 'revenue' or 'budget' is 0 and return the count of dropped rows.
#     """
#     # Count the rows where 'revenue' or 'budget' is 0
#     rows_before = df.shape[0]
#     rows_to_drop = df[(df['revenue'] == 0) | (df['budget'] == 0)]
#     rows_to_drop_count = rows_to_drop.shape[0]

#     # Drop the rows where 'revenue' or 'budget' is 0
#     df_cleaned = df[(df['revenue'] != 0) & (df['budget'] != 0)]

#     # Print the number of dropped rows
#     print(f"Number of rows dropped: {rows_to_drop_count}")
#     return df_cleaned, rows_to_drop_count

# # Example usage:
# # Assuming 'df' is your initial DataFrame with movie data
# df, dropped_rows = drop_zero_revenue_budget_and_count_dropped(df)

# # Print the result
# print(f"Updated DataFrame shape: {df.shape}")
# print(f"Rows dropped: {dropped_rows}")


In [ ]:
# import numpy as np

# def replace_zero_with_nan(df):
#     """
#     Replace rows where 'revenue' or 'budget' is 0 with NaN.
#     """
#     # Replace 'revenue' and 'budget' with NaN if they are 0
#     df['revenue'] = df['revenue'].replace(0, np.nan)
#     df['budget'] = df['budget'].replace(0, np.nan)

#     # Count how many rows were affected (i.e., rows where 'revenue' or 'budget' was 0)
#     rows_affected = df[(df['revenue'].isna()) | (df['budget'].isna())].shape[0]

#     print(f"Number of rows with 0 'revenue' or 'budget' replaced with NaN: {rows_affected}")
    
#     return df, rows_affected

# # Example usage:
# # Assuming 'df' is your initial DataFrame with movie data
# df, affected_rows = replace_zero_with_nan(df)

# # Print the result
# print(f"Updated DataFrame shape: {df.shape}")
# print(f"Rows affected: {affected_rows}")


In [57]:
functions.show_missing_values(df)


Missing Values in Columns:
id                       0
title                    0
vote_average             0
vote_count               0
revenue                  0
runtime                  0
budget                   0
imdb_id              39295
original_language        0
original_title           0
popularity               0
genres               49119
director             20609
imdb_rating          69913
imdb_votes           69913
release_year             0
language                 0
dash_title               0
dtype: int64


In [58]:
display(df)

,id,title,vote_average,vote_count,revenue,runtime,budget,imdb_id,original_language,original_title,popularity,genres,director,imdb_rating,imdb_votes,release_year,language,dash_title
0,2,Ariel,7.100,335,0,73,0,tt0094675,fi,Ariel,11.915,"comedy, drama, romance, crime",Aki Kaurismäki,7.4,8812,1988,Finnish,ariel
1,3,Shadows in Paradise,7.300,369,0,74,0,tt0092149,fi,Varjoja paratiisissa,16.287,"comedy, drama, romance",Aki Kaurismäki,7.5,7587,1986,Finnish,shadows-in-paradise
2,5,Four Rooms,5.800,2628,4257354,98,4000000,tt0113101,en,Four Rooms,21.312,comedy,"Quentin Tarantino, Robert Rodriguez, Alexandre...",6.7,112798,1995,English,four-rooms
3,6,Judgment Night,6.500,331,12136938,109,21000000,tt0107286,en,Judgment Night,8.924,"action, crime, thriller",Stephen Hopkins,6.6,19361,1993,English,judgment-night
6,11,Star Wars,8.203,20622,775398007,121,11000000,tt0076759,en,Star Wars,98.770,"adventure, action, science fiction",George Lucas,8.6,1482739,1977,English,star-wars
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027797,1399927,The Honeymooners Christmas Special,0.000,0,0,49,0,tt0445439,en,The Honeymooners Christmas Special,0.600,comedy,Jackie Gleason,8.1,78,1978,English,the-honeymooners-christmas-special
1027869,1400003,Fly High,0.000,0,0,120,0,NaN,mn,Дүүлэн нис,0.000,"drama, history, romance",Dorjsuren Shadav,NaN,<NA>,2013,Mongolian,fly-high
1027897,1400034,The Peony Pavilion,0.000,0,0,122,0,NaN,en,The Peony Pavilion,0.000,None,Ying Fang,NaN,<NA>,1986,English,the-peony-pavilion
1027917,1400057,Turksib,0.000,0,0,73,0,tt0275745,ru,На перевале,0.000,drama,"Serik Zharmukhamedov, Kadyr Zhetpisbay",NaN,<NA>,1986,Russian,turksib


In [59]:
rename_columns = {
    'id': 'tmdb_id',
    'vote_average': 'tmdb_rating',
    'vote_count': 'tmdb_votes'
}

df.rename(columns=rename_columns, inplace=True)

In [60]:
df['popularity'] = df['popularity'].round(1)
df['tmdb_rating'] = df['tmdb_rating'].round(1)

display(df)

,tmdb_id,title,tmdb_rating,tmdb_votes,revenue,runtime,budget,imdb_id,original_language,original_title,popularity,genres,director,imdb_rating,imdb_votes,release_year,language,dash_title
0,2,Ariel,7.1,335,0,73,0,tt0094675,fi,Ariel,11.9,"comedy, drama, romance, crime",Aki Kaurismäki,7.4,8812,1988,Finnish,ariel
1,3,Shadows in Paradise,7.3,369,0,74,0,tt0092149,fi,Varjoja paratiisissa,16.3,"comedy, drama, romance",Aki Kaurismäki,7.5,7587,1986,Finnish,shadows-in-paradise
2,5,Four Rooms,5.8,2628,4257354,98,4000000,tt0113101,en,Four Rooms,21.3,comedy,"Quentin Tarantino, Robert Rodriguez, Alexandre...",6.7,112798,1995,English,four-rooms
3,6,Judgment Night,6.5,331,12136938,109,21000000,tt0107286,en,Judgment Night,8.9,"action, crime, thriller",Stephen Hopkins,6.6,19361,1993,English,judgment-night
6,11,Star Wars,8.2,20622,775398007,121,11000000,tt0076759,en,Star Wars,98.8,"adventure, action, science fiction",George Lucas,8.6,1482739,1977,English,star-wars
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027797,1399927,The Honeymooners Christmas Special,0.0,0,0,49,0,tt0445439,en,The Honeymooners Christmas Special,0.6,comedy,Jackie Gleason,8.1,78,1978,English,the-honeymooners-christmas-special
1027869,1400003,Fly High,0.0,0,0,120,0,NaN,mn,Дүүлэн нис,0.0,"drama, history, romance",Dorjsuren Shadav,NaN,<NA>,2013,Mongolian,fly-high
1027897,1400034,The Peony Pavilion,0.0,0,0,122,0,NaN,en,The Peony Pavilion,0.0,None,Ying Fang,NaN,<NA>,1986,English,the-peony-pavilion
1027917,1400057,Turksib,0.0,0,0,73,0,tt0275745,ru,На перевале,0.0,drama,"Serik Zharmukhamedov, Kadyr Zhetpisbay",NaN,<NA>,1986,Russian,turksib


In [61]:
df['clean_title'] = functions.prepare_clean_titles(df, 'title')

In [63]:
display(df)

,tmdb_id,title,tmdb_rating,tmdb_votes,revenue,runtime,budget,imdb_id,original_language,original_title,popularity,genres,director,imdb_rating,imdb_votes,release_year,language,dash_title,clean_title
0,2,Ariel,7.1,335,0,73,0,tt0094675,fi,Ariel,11.9,"comedy, drama, romance, crime",Aki Kaurismäki,7.4,8812,1988,Finnish,ariel,ariel
1,3,Shadows in Paradise,7.3,369,0,74,0,tt0092149,fi,Varjoja paratiisissa,16.3,"comedy, drama, romance",Aki Kaurismäki,7.5,7587,1986,Finnish,shadows-in-paradise,shadows in paradise
2,5,Four Rooms,5.8,2628,4257354,98,4000000,tt0113101,en,Four Rooms,21.3,comedy,"Quentin Tarantino, Robert Rodriguez, Alexandre...",6.7,112798,1995,English,four-rooms,four rooms
3,6,Judgment Night,6.5,331,12136938,109,21000000,tt0107286,en,Judgment Night,8.9,"action, crime, thriller",Stephen Hopkins,6.6,19361,1993,English,judgment-night,judgment night
6,11,Star Wars,8.2,20622,775398007,121,11000000,tt0076759,en,Star Wars,98.8,"adventure, action, science fiction",George Lucas,8.6,1482739,1977,English,star-wars,star wars
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027797,1399927,The Honeymooners Christmas Special,0.0,0,0,49,0,tt0445439,en,The Honeymooners Christmas Special,0.6,comedy,Jackie Gleason,8.1,78,1978,English,the-honeymooners-christmas-special,the honeymooners christmas special
1027869,1400003,Fly High,0.0,0,0,120,0,NaN,mn,Дүүлэн нис,0.0,"drama, history, romance",Dorjsuren Shadav,NaN,<NA>,2013,Mongolian,fly-high,fly high
1027897,1400034,The Peony Pavilion,0.0,0,0,122,0,NaN,en,The Peony Pavilion,0.0,None,Ying Fang,NaN,<NA>,1986,English,the-peony-pavilion,the peony pavilion
1027917,1400057,Turksib,0.0,0,0,73,0,tt0275745,ru,На перевале,0.0,drama,"Serik Zharmukhamedov, Kadyr Zhetpisbay",NaN,<NA>,1986,Russian,turksib,turksib


In [64]:
new_column_order = [
    'title', 'clean_title', 'original_title', 'genres', 'director', 'release_year',
    'runtime', 'budget', 'revenue', 'popularity', 'tmdb_rating', 'tmdb_votes', 
    'imdb_rating', 'imdb_votes', 'language', 'tmdb_id', 'imdb_id'
]

# Ensure all columns are included, add any missing ones from the original DataFrame
final_column_order = [col for col in new_column_order if col in df.columns]
missing_columns = [col for col in df.columns if col not in final_column_order]
final_column_order.extend(missing_columns)

# Reorder the DataFrame
df = df[final_column_order]

In [65]:
df = df.sort_values(by='tmdb_id').reset_index(drop=True)

df.to_csv('../data/local/clean/films_before19.csv', index=False)

In [41]:
# import re
# import time
# import pandas as pd
# from bs4 import BeautifulSoup
# from urllib.parse import quote
# import requests

# def get_letterboxd_movie_details_by_dash_title(dash_title):
#     """
#     Fetch movie details by dash_title from Letterboxd.
#     """
#     base_url = f"https://letterboxd.com/film/{quote(dash_title)}/"
#     try:
#         response = requests.get(base_url, timeout=10)
#         if response.status_code == 200:
#             return response.text
#         elif response.status_code == 404:
#             print(f"404 Not Found: {dash_title}")
#         else:
#             print(f"Failed to fetch: {dash_title}, Status Code: {response.status_code}")
#         return None
#     except requests.RequestException as e:
#         print(f"Request error for {dash_title}: {e}")
#         return None

# def extract_movie_info_from_html(html_content):
#     """
#     Extract director, genres, release year, and runtime from the movie's HTML content.
#     """
#     soup = BeautifulSoup(html_content, 'html.parser')

#     # Debug: Print page title for verification
#     page_title = soup.title.string if soup.title else "No Title"
#     print(f"Page Title: {page_title}")

#     # Extract Year (in parentheses, must be 4 digits)
#     release_year = None
#     year_match = re.search(r'\((\d{4})\)', page_title)
#     if year_match:
#         release_year = year_match.group(1)
    
#     # Extract Director (after "directed by")
#     director = None
#     if "directed by" in page_title:
#         try:
#             director = page_title.split("directed by")[1].split("•")[0].strip()
#         except IndexError:
#             print(f"Failed to parse director from title: {page_title}")
    
#     # Fallback: Check the movie details section for director
#     if not director:
#         director_tag = soup.find('span', class_='director')
#         if director_tag:
#             director = director_tag.get_text(strip=True)

#     # Genres
#     genres = []
#     genre_tags = soup.select('a[href*="/films/by/genre/"]')
#     if genre_tags:
#         genres = [genre.get_text(strip=True) for genre in genre_tags]

#     # Runtime
#     runtime = None
#     runtime_tag = soup.select_one('p.text-link.text-footer')
#     if runtime_tag and 'mins' in runtime_tag.text:
#         runtime = ''.join(filter(str.isdigit, runtime_tag.get_text(strip=True)))

#     print(f"Extracted: Director={director}, Genres={genres}, Year={release_year}, Runtime={runtime}")
#     return director, genres, release_year, runtime

# def fill_missing_movie_data(df, max_titles=10, backup_file='backup_df.csv', sleep_time=2):
#     # Set to track processed titles
#     processed_titles = set()

#     # Filter rows where any field is missing or contains an empty value
#     rows_to_update = df[df['director'].isna() | df['release_year'].isna() | df['runtime'].isna() | 
#                         (df['director'] == '') | (df['release_year'] == '') | (df['runtime'] == '')]

#     # Limit the number of rows to process
#     rows_to_update = rows_to_update.iloc[:max_titles]

#     # Counter for updated rows
#     rows_updated = 0

#     # Loop through the filtered rows with missing data
#     for index, row in rows_to_update.iterrows():
#         dash_title = row['dash_title']

#         # Skip already processed titles
#         if dash_title not in processed_titles:
#             print(f"Processing dash_title: {dash_title}")

#             html_content = get_letterboxd_movie_details_by_dash_title(dash_title)
#             if html_content:
#                 director, genres, release_year, runtime = extract_movie_info_from_html(html_content)

#                 # Track changes for this row
#                 updated_fields = []

#                 # Check if each field is missing and update accordingly
#                 if pd.isna(row['director']) or row['director'] == '':
#                     if director:  # Update only if there's a valid director
#                         df.at[index, 'director'] = director
#                         updated_fields.append('director')

#                 if pd.isna(row['release_year']) or row['release_year'] == '':
#                     if release_year:  # Update only if there's a valid release_year
#                         df.at[index, 'release_year'] = release_year
#                         updated_fields.append('release_year')

#                 if pd.isna(row['runtime']) or row['runtime'] == '':
#                     if runtime:  # Update only if there's a valid runtime
#                         df.at[index, 'runtime'] = runtime
#                         updated_fields.append('runtime')

#                 # If any field was updated, increment the counter and print changes
#                 if updated_fields:
#                     rows_updated += 1
#                     print(f"Row {index} updated: {', '.join(updated_fields)}")
#                 else:
#                     print(f"No updates for Row {index} (dash_title: {dash_title})")

#                 # Mark this movie title as processed
#                 processed_titles.add(dash_title)
#             else:
#                 print(f"Failed to fetch content for {dash_title}")

#             # Sleep between requests to avoid hitting the server too fast
#             time.sleep(sleep_time)

#         # Save or append a backup file after each update
#         if rows_updated % 10 == 0 or rows_updated == len(rows_to_update):
#             if backup_file:
#                 df.to_csv(backup_file, index=False)
#                 print(f"Backup saved to {backup_file} after {rows_updated} updates.")

#     # Print summary of updates
#     print(f"\nTotal rows updated: {rows_updated}")
#     return df


# # Fill missing data for 50 titles with a backup
# test_df = fill_missing_movie_data(df, max_titles=10, backup_file='movies_backup_debug.csv', sleep_time=2)

# # Print the updated DataFrame
# print(test_df[['dash_title', 'director', 'release_year', 'runtime']])


In [42]:
# import re
# import time
# import pandas as pd
# from bs4 import BeautifulSoup
# from urllib.parse import quote
# import requests

# def get_letterboxd_movie_details_by_dash_title(dash_title):
#     """
#     Fetch movie details by dash_title from Letterboxd.
#     """
#     base_url = f"https://letterboxd.com/film/{quote(dash_title)}/"
#     try:
#         response = requests.get(base_url, timeout=10)
#         if response.status_code == 200:
#             return response.text
#         elif response.status_code == 404:
#             print(f"404 Not Found: {dash_title}")
#         else:
#             print(f"Failed to fetch: {dash_title}, Status Code: {response.status_code}")
#         return None
#     except requests.RequestException as e:
#         print(f"Request error for {dash_title}: {e}")
#         return None

# def extract_movie_info_from_html(html_content):
#     """
#     Extract director, release year, and runtime from the movie's HTML content.
#     """
#     soup = BeautifulSoup(html_content, 'html.parser')

#     # Debug: Print page title for verification
#     page_title = soup.title.string if soup.title else "No Title"
#     print(f"Page Title: {page_title}")

#     # Extract Year (in parentheses, must be 4 digits)
#     release_year = None
#     year_match = re.search(r'\((\d{4})\)', page_title)
#     if year_match:
#         release_year = year_match.group(1)
    
#     # Extract Director (after "directed by")
#     director = None
#     if "directed by" in page_title:
#         try:
#             director = page_title.split("directed by")[1].split("•")[0].strip()
#         except IndexError:
#             print(f"Failed to parse director from title: {page_title}")
    
#     # Fallback: Check the movie details section for director
#     if not director:
#         director_tag = soup.find('span', class_='director')
#         if director_tag:
#             director = director_tag.get_text(strip=True)

#     # Runtime
#     runtime = None
#     runtime_tag = soup.select_one('p.text-link.text-footer')
#     if runtime_tag and 'mins' in runtime_tag.text:
#         runtime = ''.join(filter(str.isdigit, runtime_tag.get_text(strip=True)))

#     print(f"Extracted: Director={director}, Year={release_year}, Runtime={runtime}")
#     return director, release_year, runtime

# def fill_missing_movie_data(df, max_titles=10, backup_file='backup_df.csv', sleep_time=2):
#     # Set to track processed titles
#     processed_titles = set()

#     # Filter rows where any field is missing or contains an empty value
#     rows_to_update = df[df['director'].isna() | df['release_year'].isna() | df['runtime'].isna() | 
#                         (df['director'] == '') | (df['release_year'] == '') | (df['runtime'] == '')]

#     # Limit the number of rows to process
#     rows_to_update = rows_to_update.iloc[:max_titles]

#     # Counter for updated rows
#     rows_updated = 0

#     # Loop through the filtered rows with missing data
#     for index, row in rows_to_update.iterrows():
#         dash_title = row['dash_title']

#         # Skip already processed titles
#         if dash_title not in processed_titles:
#             print(f"Processing dash_title: {dash_title}")

#             html_content = get_letterboxd_movie_details_by_dash_title(dash_title)
#             if html_content:
#                 director, release_year, runtime = extract_movie_info_from_html(html_content)

#                 # Track changes for this row
#                 updated_fields = []

#                 # Check if each field is missing and update accordingly
#                 if pd.isna(row['director']) or row['director'] == '':
#                     if director:  # Update only if there's a valid director
#                         df.at[index, 'director'] = director
#                         updated_fields.append('director')

#                 if pd.isna(row['release_year']) or row['release_year'] == '':
#                     if release_year:  # Update only if there's a valid release_year
#                         df.at[index, 'release_year'] = release_year
#                         updated_fields.append('release_year')

#                 if pd.isna(row['runtime']) or row['runtime'] == '':
#                     if runtime:  # Update only if there's a valid runtime
#                         df.at[index, 'runtime'] = runtime
#                         updated_fields.append('runtime')

#                 # If any field was updated, increment the counter and print changes
#                 if updated_fields:
#                     rows_updated += 1
#                     print(f"Row {index} updated: {', '.join(updated_fields)}")
#                 else:
#                     print(f"No updates for Row {index} (dash_title: {dash_title})")

#                 # Mark this movie title as processed
#                 processed_titles.add(dash_title)
#             else:
#                 print(f"Failed to fetch content for {dash_title}")

#             # Sleep between requests to avoid hitting the server too fast
#             time.sleep(sleep_time)

#         # Save or append a backup file after each update
#         if rows_updated % 10 == 0 or rows_updated == len(rows_to_update):
#             if backup_file:
#                 df.to_csv(backup_file, index=False)
#                 print(f"Backup saved to {backup_file} after {rows_updated} updates.")

#     # Print summary of updates
#     print(f"\nTotal rows updated: {rows_updated}")
#     return df


# # Fill missing data for 10 titles with a backup
# test_df = fill_missing_movie_data(df, max_titles=10, backup_file='movies_backup_debug.csv', sleep_time=2)

# # Print the updated DataFrame
# print(test_df[['dash_title', 'director', 'release_year', 'runtime']])


In [ ]:
# import re
# import time
# import pandas as pd
# from bs4 import BeautifulSoup
# from urllib.parse import quote
# import requests

# def get_letterboxd_movie_details_by_dash_title(dash_title):
#     """
#     Fetch movie details by dash_title from Letterboxd.
#     """
#     base_url = f"https://letterboxd.com/film/{quote(dash_title)}/"
#     try:
#         response = requests.get(base_url, timeout=10)
#         if response.status_code == 200:
#             return response.text
#         elif response.status_code == 404:
#             print(f"404 Not Found: {dash_title}")
#         else:
#             print(f"Failed to fetch: {dash_title}, Status Code: {response.status_code}")
#         return None
#     except requests.RequestException as e:
#         print(f"Request error for {dash_title}: {e}")
#         return None

# def extract_movie_info_from_html(html_content):
#     """
#     Extract director, release year, and runtime from the movie's HTML content.
#     """
#     soup = BeautifulSoup(html_content, 'html.parser')

#     # Debug: Print page title for verification
#     page_title = soup.title.string if soup.title else "No Title"
#     print(f"Page Title: {page_title}")

#     # Extract Year (in parentheses, must be 4 digits)
#     release_year = None
#     year_match = re.search(r'\((\d{4})\)', page_title)
#     if year_match:
#         release_year = year_match.group(1)
    
#     # Extract Director (after "directed by")
#     director = None
#     if "directed by" in page_title:
#         try:
#             director = page_title.split("directed by")[1].split("•")[0].strip()
#         except IndexError:
#             print(f"Failed to parse director from title: {page_title}")
    
#     # Fallback: Check the movie details section for director
#     if not director:
#         director_tag = soup.find('span', class_='director')
#         if director_tag:
#             director = director_tag.get_text(strip=True)

#     # Runtime
#     runtime = None
#     runtime_tag = soup.select_one('p.text-link.text-footer')
#     if runtime_tag and 'mins' in runtime_tag.text:
#         runtime = ''.join(filter(str.isdigit, runtime_tag.get_text(strip=True)))

#     print(f"Extracted: Director={director}, Year={release_year}, Runtime={runtime}")
#     return director, release_year, runtime

# def fill_missing_movie_data(df, backup_file='backup_df.csv', sleep_time=2):
#     # Set to track processed titles
#     processed_titles = set()

#     # Filter rows where any field is missing or contains an empty value
#     rows_to_update = df[df['director'].isna() | df['release_year'].isna() | df['runtime'].isna() | 
#                         (df['director'] == '') | (df['release_year'] == '') | (df['runtime'] == '')]

#     # Counter for updated rows
#     rows_updated = 0

#     # Loop through the filtered rows with missing data
#     for index, row in rows_to_update.iterrows():
#         dash_title = row['dash_title']

#         # Skip already processed titles
#         if dash_title not in processed_titles:
#             print(f"Processing dash_title: {dash_title}")

#             html_content = get_letterboxd_movie_details_by_dash_title(dash_title)
#             if html_content:
#                 director, release_year, runtime = extract_movie_info_from_html(html_content)

#                 # Track changes for this row
#                 updated_fields = []

#                 # Add new columns if data is extracted
#                 if director:
#                     df.at[index, 'extracted_director'] = director
#                     updated_fields.append('extracted_director')

#                 if release_year:
#                     df.at[index, 'extracted_release_year'] = release_year
#                     updated_fields.append('extracted_release_year')

#                 if runtime:
#                     df.at[index, 'extracted_runtime'] = runtime
#                     updated_fields.append('extracted_runtime')

#                 # If any field was updated, increment the counter and print changes
#                 if updated_fields:
#                     rows_updated += 1
#                     print(f"Row {index} updated: {', '.join(updated_fields)}")
#                 else:
#                     print(f"No updates for Row {index} (dash_title: {dash_title})")

#                 # Mark this movie title as processed
#                 processed_titles.add(dash_title)
#             else:
#                 print(f"Failed to fetch content for {dash_title}")

#             # Sleep between requests to avoid hitting the server too fast
#             time.sleep(sleep_time)

#         # Save or append a backup file after each update
#         if rows_updated % 10 == 0 or rows_updated == len(rows_to_update):
#             if backup_file:
#                 df.to_csv(backup_file, index=False)
#                 print(f"Backup saved to {backup_file} after {rows_updated} updates.")

#     # Print summary of updates
#     print(f"\nTotal rows updated: {rows_updated}")
#     return df


# # Example usage
# # Assuming 'df' is your initial DataFrame with missing movie data
# # Fill missing data for the entire DataFrame with a backup
# test_df = fill_missing_movie_data(df, backup_file='movies_backup_debug.csv', sleep_time=2)

# # Print the updated DataFrame
# print(test_df[['dash_title', 'extracted_director', 'extracted_release_year', 'extracted_runtime']])


In [ ]:
# import os
# import requests
# import pandas as pd
# from dotenv import load_dotenv

# # Load environment variables from the .env file
# load_dotenv()

# # Retrieve TMDB API key from environment
# TMDB_API_KEY = os.getenv('TMDB_API_KEY')

# def fetch_movie_details_from_tmdb(movie_id):
#     """
#     Fetch movie details from TMDB API using the movie ID.
#     """
#     base_url = f"https://api.themoviedb.org/3/movie/{movie_id}"
#     params = {
#         'api_key': TMDB_API_KEY,
#         'language': 'en-US'
#     }
#     try:
#         response = requests.get(base_url, params=params, timeout=10)
#         if response.status_code == 200:
#             return response.json()
#         else:
#             print(f"Failed to fetch details for movie ID {movie_id}: {response.status_code}")
#             return None
#     except requests.RequestException as e:
#         print(f"Request error for movie ID {movie_id}: {e}")
#         return None

# def update_movie_data(df, batch_size=100, backup_file="movie_data_backup.csv"):
#     """
#     Update missing movie data in the DataFrame by making requests to the TMDB API.
#     Process the data in batches and append the progress to a backup file.
#     """
#     rows_updated = 0
#     rows_failed = 0
#     total_rows = len(df)
    
#     # Initialize new columns if they don't exist
#     if 'new_revenue' not in df.columns:
#         df['new_revenue'] = None
#     if 'new_budget' not in df.columns:
#         df['new_budget'] = None
#     if 'new_genres' not in df.columns:
#         df['new_genres'] = None
#     if 'new_release_year' not in df.columns:
#         df['new_release_year'] = None
    
#     # Process the DataFrame in batches
#     for start_idx in range(0, total_rows, batch_size):
#         end_idx = min(start_idx + batch_size, total_rows)
#         print(f"\nProcessing batch {start_idx // batch_size + 1} ({start_idx + 1}-{end_idx} of {total_rows})...")
        
#         batch_df = df.iloc[start_idx:end_idx]
        
#         # Iterate over the rows in the batch
#         for index, row in batch_df.iterrows():
#             # Check if any of the target columns ('revenue', 'budget', 'genres', 'release_year') are missing
#             if pd.isna(row['revenue']) or pd.isna(row['budget']) or pd.isna(row['genres']) or pd.isna(row['release_year']):
#                 movie_id = row['id']
#                 print(f"Fetching data for movie ID {movie_id}...")
                
#                 # Fetch movie details from TMDB
#                 movie_details = fetch_movie_details_from_tmdb(movie_id)

#                 if movie_details:
#                     updated = False
#                     # Update 'revenue' if missing
#                     if pd.isna(row['revenue']) and 'revenue' in movie_details:
#                         df.at[index, 'new_revenue'] = movie_details['revenue']
#                         updated = True

#                     # Update 'budget' if missing
#                     if pd.isna(row['budget']) and 'budget' in movie_details:
#                         df.at[index, 'new_budget'] = movie_details['budget']
#                         updated = True

#                     # Update 'release_year' if missing
#                     if pd.isna(row['release_year']) and 'release_date' in movie_details:
#                         df.at[index, 'new_release_year'] = movie_details['release_date'].split('-')[0]
#                         updated = True

#                     # Update 'genres' if missing
#                     if pd.isna(row['genres']) and 'genres' in movie_details:
#                         genres = movie_details.get('genres', [])
#                         genre_names = [genre['name'] for genre in genres]
#                         df.at[index, 'new_genres'] = ', '.join(genre_names) if genre_names else None
#                         updated = True

#                     # Track the number of updates
#                     if updated:
#                         rows_updated += 1
#                     else:
#                         rows_failed += 1
#                 else:
#                     rows_failed += 1

#         # Append the updated batch to the backup file after each batch
#         # If the file doesn't exist, it will be created. If it does exist, new rows will be appended.
#         df.to_csv(backup_file, mode='a', header=not os.path.exists(backup_file), index=False)
#         print(f"Batch {start_idx // batch_size + 1} processed and appended to {backup_file}.")
    
#     # Print summary of updates
#     print(f"\nTotal successful updates: {rows_updated}")
#     print(f"Total failed fetches: {rows_failed}")
#     return df


# # Example: Assuming 'df' is your existing DataFrame (already loaded from a CSV, database, etc.)
# # Run the update function (in batches of 1000 rows, for example)
# updated_df = update_movie_data(df, batch_size=100, backup_file="movie_data_backup.csv")

# # Print the updated DataFrame with the new columns after processing all batches
# print(updated_df[['id', 'new_revenue', 'new_budget', 'new_genres', 'new_release_year']])


In [ ]:
test_df.head(5)

In [ ]:
df stop_code

In [ ]:
df.head(5)

In [ ]:
df derp # stop code here

### New DF from a year span
- Titles between 2019 and 2024
- Drop empty rows in columns 'genre', 'director', 'original_title', and 'title'
- 174702 rows × 17 columns

In [ ]:
# Filter rows where 'release_year' is between 2019 and 2024
filtered_df = df[(df['release_year'] >= 2019) & (df['release_year'] <= 2024)]

# Drop rows where 'genres' or 'director' have empty or missing values
filtered_df = filtered_df.dropna(subset=['genres', 'director', 'original_title', 'title'])


In [ ]:
functions.show_missing_values(filtered_df)

In [ ]:
display(filtered_df)

In [ ]:
movie_df = filtered_df.copy()

In [ ]:
# drop columns
columns_to_drop = ['release_date']

movie_df = movie_df.drop(columns=columns_to_drop)

In [ ]:
movie_df['clean_title'] = functions.prepare_clean_titles(movie_df, 'title')

# # reorder columns
# movie_df = movie_df[['id', 'title', 'original_title', 'clean_title', 'release_year', 'imdb_id', 'imdb_rating', 'imdb_votes', 'genres', 'director', 'revenue', 'budget', 'runtime', 'original_language', 'popularity']]

In [ ]:
# clean genres
movie_df['genres'] = functions.clean_genres(movie_df, 'genres')
movie_df.head()

In [ ]:
columns_to_convert = ['imdb_votes', 'revenue', 'budget', 'runtime', 'vote_count']

# Check if the columns exist in the DataFrame
columns_to_convert = [col for col in columns_to_convert if col in movie_df.columns]

# Convert the specified columns to 'Int64' type, handling errors gracefully
movie_df[columns_to_convert] = movie_df[columns_to_convert].apply(pd.to_numeric, errors='coerce').astype('Int64')

# Now the DataFrame should have the correct data types
print(movie_df.head())


In [ ]:
# # Check empty rows
# total_rows = len(movie_df)

# empty_rows = movie_df.isna().any(axis=1).sum()

# print(f'Total number of rows:\n{total_rows}')
# print(f'\nNumber of rows with empty values:\n{empty_rows}')

In [ ]:
movie_df['language'] = movie_df['original_language'].apply(functions.get_language_name)

movie_df.head()

Handle unknown languages

In [ ]:
print(f'Unique values in language column:\n{movie_df['language'].unique()}')

In [ ]:
print(f'Value counts in language column:\n{movie_df['language'].value_counts()}')

In [ ]:
# occurrences of [cn]
unknown_lang = movie_df[movie_df['original_language'] == 'cn']
# print(unknown_lang)

In [ ]:
# replace [cn] with a proper label
movie_df['language'] = movie_df['language'].replace('Unknown language [cn]', 'Cantonese')

In [ ]:
# occurrences of xx
unknown_lang = movie_df[movie_df['original_language'] == 'xx']
# print(unknown_lang)

In [ ]:
# replace [xx] with a proper label
movie_df['language'] = movie_df['language'].replace('Unknown language [xx]', 'Unknown')

In [ ]:
print(f'Unique values in language column after re labeling: {movie_df['language'].unique()}')

In [ ]:
movie_df.drop(columns=['original_language'], inplace=True)

In [ ]:
movie_df.head(50)

Popularity to one decimal

In [ ]:
movie_df['popularity'] = movie_df['popularity'].round(1)
movie_df['vote_average'] = movie_df['vote_average'].round(1)

display(movie_df)

In [ ]:
functions.show_missing_values(movie_df)

In [ ]:
display(movie_df)

#### Check runtime
- Movies with less than 40' runtime are considered short films.
- There's likely a lot of missing values.

In [ ]:
rows_to_remove = movie_df[(movie_df['runtime'] >= 1) & (movie_df['runtime'] <= 40)]

removed_count = rows_to_remove.shape[0]

movie_df = movie_df[~((movie_df['runtime'] >= 1) & (movie_df['runtime'] <= 40))]

print(f'Number of rows removed: {removed_count}')

In [ ]:
display(movie_df)

#### Rename and reorder columns

In [ ]:
rename_columns = {
    'id': 'tmdb_id',
    'vote_average': 'tmdb_rating',
    'vote_count': 'tmdb_votes'
}

movie_df.rename(columns=rename_columns, inplace=True)

In [ ]:
new_column_order = [
    'title', 'clean_title', 'original_title', 'genres', 'director', 'release_year',
    'runtime', 'budget', 'revenue', 'popularity', 'tmdb_rating', 'tmdb_votes', 
    'imdb_rating', 'imdb_votes', 'language', 'tmdb_id', 'imdb_id'
]

# Reorder the DataFrame columns
movie_df = movie_df[new_column_order]


In [ ]:
display(movie_df)

#### df to csv

In [ ]:
movie_df = movie_df.sort_values(by='tmdb_id').reset_index(drop=True)

movie_df.to_csv('../data/local/clean/films_19to24.csv', index=False)

### Top 10s

In [ ]:
# most popular genres:
# split column by commas
df_exploded_genres = movie_df['genres'].str.split(',').explode().str.strip()

# add column for popularity
df_genres_popularity = df_exploded_genres.to_frame(name='genre').join(movie_df['popularity'])

# calculate average popularity
genre_popularity = df_genres_popularity.groupby('genre')['popularity'].mean().sort_values(ascending=False)

print('Most Popular Genres:')
print(genre_popularity.head(10))

# calculate average popularity
language_popularity = movie_df.groupby('language')['popularity'].mean().sort_values(ascending=False)

print('\nMost Popular Languages:')
print(language_popularity.head(10))

In [ ]:
# split 'genres' by commas, explode it, create a row for each genre
df_exploded_genres = movie_df['genres'].str.split(',').explode().str.strip()

# 'popularity' column to exploded genres
df_genres_imdb_rating = df_exploded_genres.to_frame(name='genre').join(movie_df['imdb_rating'])

# calculate average imdb_rating
genre_imdb_rating = df_genres_imdb_rating.groupby('genre')['imdb_rating'].mean().sort_values(ascending=False)

print('Most Popular Genres:')
print(genre_imdb_rating.head(10))

# most popular languages:
# calculate average imdb_rating
language_imdb_rating = movie_df.groupby('language')['imdb_rating'].mean().sort_values(ascending=False)

print('\nMost Popular Languages:')
print(language_popularity.head(10))

### Correlation Plots

In [ ]:
# correlations for numeric columns
numeric_columns = ['popularity', 'revenue', 'budget', 'runtime', 'imdb_rating', 'imdb_votes']
correlation_matrix = movie_df[numeric_columns].corr()

# correlation matrix heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='winter', vmin=-1, vmax=1)
plt.title('Correlation Heatmap of Numeric Columns')
plt.show()

In [ ]:
# most popular genres
# split and explode the genres column
df_exploded_genres = movie_df['genres'].str.split(',').explode().str.strip()
df_genres_popularity = df_exploded_genres.to_frame(name='genre').join(movie_df['popularity'])

# group and calculate average popularity
genre_popularity = df_genres_popularity.groupby('genre')['popularity'].mean().sort_values(ascending=False)
top_20_genres = genre_popularity.head(20)

top_20_genres.index = top_20_genres.index.str.title()

# top 20 plot
plt.figure(figsize=(10, 8))
sns.barplot(x=top_20_genres.values, y=top_20_genres.index, palette='viridis')
plt.title('Top 20 Most Popular Genres (2018)')
plt.xlabel('Average Popularity')
plt.ylabel('Genre')
plt.show()

In [ ]:
# most popular languages
language_popularity = movie_df.groupby('language')['popularity'].mean().sort_values(ascending=False)
top_10_languages = language_popularity.head(10)

plt.figure(figsize=(10, 6))
sns.barplot(x=top_10_languages.values, y=top_10_languages.index, palette='magma')
plt.title('Top 10 Most Popular Languages (2018)')
plt.xlabel('Average Popularity')
plt.ylabel('Language')
plt.show()